# Sonification of Spectograms (work in progress)

In [ ]:
import librosa
import librosa.display
import numpy as np
import os
from IPython import display as ipd
import matplotlib.pyplot as plt

%matplotlib inline

## Sound Example

In [ ]:
x, Fs = librosa.load(os.path.join('data_audio','samples','01Pia1F060f_np___0.wav'))
print('Original Audio:')
ipd.display(ipd.Audio(x, rate=Fs))

## Performing STFT

In [ ]:
N = 2048
H = 64

color = 'gray_r' 

X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, window='hann', pad_mode='constant', center=True)

Y = np.log(1 + 100 * np.abs(X) ** 2)

T_coef = np.arange(X.shape[1]) * H / Fs
T_coef_librosa = librosa.frames_to_time(np.arange(X.shape[1]), sr=Fs, hop_length=H)

K = N // 2
F_coef = np.arange(K+1) * Fs / N
F_coef_librosa = librosa.fft_frequencies(sr=Fs, n_fft=N)

plt.figure(figsize=(6, 3))
extent = [T_coef[0], T_coef[-1], F_coef[0], F_coef[-1]]
plt.imshow(Y, cmap=color, aspect='auto', origin='lower', extent=extent)
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
print(T_coef[0:5], (T_coef[1]-T_coef[0])*Fs)
plt.plot(Y[300,:]) 

In [ ]:
print(Y.shape)

In [ ]:
from libsoni.core.methods import generate_sinusoid

In [ ]:
import numpy as np
from typing import Tuple

from libsoni.util.utils import normalize_signal, fade_signal, smooth_weights

def sonify_spectrum(spectogram: np.ndarray,
                    frequency_coef: np.ndarray,
                    time_coef: np.ndarray,
                    fade_duration: float = 0.05,
                    sonification_duration: int = None,
                    normalize: bool = True,
                    fs: int = 22050,
                    tuning_frequency: float = 440.0) -> np.ndarray:
    
    """Sonify chromagram

        Parameters
        ----------
        spectogram: np.ndarray
            Magnitude spectogram to be sonified.
        frequency_coef: np.ndarray
            Array containing frequencies for sonification.
        time_coef: np.ndarray,
            Array containing time coefficients, in seconds, for sonification.
        sonification_duration: int, default = None
            Duration of audio, given in samples
        fade_duration: float, default = 0.05
            Duration of fade-in and fade-out at beginning and end of the sonification, given in seconds.
        fs: int, default: 44100
            sampling rate in Samples/second
        normalize: bool, default = True
            Decides, if output signal is normalized to [-1,1].
        fs: int, default = 22050
            Sampling rate, in samples per seconds.
        tuning_frequency : float, default = 440
            Tuning frequency.

        Returns
        -------
            y: synthesized tone
        """
    assert spectogram.shape[0] == len(frequency_coef), f'The length of frequency_coef must match spectogram.shape[0]'
    assert spectogram.shape[1] == len(time_coef), f'The length of time_coef must match spectogram.shape[1]'

    
    # Determine length of sonification
    num_samples = sonification_duration if sonification_duration is not None else int(time_coef[-1] * fs)

    # Initialize sonification
    spectogram_sonification = np.zeros(num_samples)
    
    for i in range(spectogram.shape[0]):
        weighting_vector = spectogram[i,:]
        
        t_diff = T_coef[1:]-T_coef[0:-1]
        
        weighting_vector = np.repeat(spectogram[i, :], int((T_coef[1]-T_coef[0])*fs))
        weighting_vector 
        
        print(len(weighting_vector))
        weighting_vector_smoothed = smooth_weights(weights=weighting_vector, fading_samples=int(int(T_coef[1]-T_coef[0])*Fs / 8))

        sinusoid = generate_sinusoid(frequency=F_coef[i],
                                     phase=0,
                                     duration_sec=num_samples / fs,
                                     fs=fs)
        print(len(sinusoid))

        spectogram_sonification += (sinusoid * weighting_vector_smoothed)

    spectogram_sonification = fade_signal(spectogram_sonification, fs=fs, fading_sec=fade_duration)

    spectogram_sonification = normalize_signal(spectogram_sonification) if normalize else spectogram_sonification

    return chroma_sonification

In [ ]:
t_diff = (T_coef[1:]-T_coef[0:-1])*Fs

int(T_coef[-1] * fs)
print(t_diff)

print(len(t_diff)*64)

print(T_coef[-1]*Fs)

Y_soni = sonify_spectrum(Y, F_coef, T_coef)


## Sonification via Inverse STFT

In [ ]:
def sonify_spectrum_via_istft(stft, hop_size):
    return librosa.istft(stft, hop_length=hop_size)

In [ ]:
Y = np.angle(X)
sonified_spectrum = sonify_spectrum_via_istft(Y, H)

print('Sonified Spectrum:')
ipd.display(ipd.Audio(sonified_spectrum, rate=Fs))

## Sonification via column-wise inverse FFT

In [ ]:
def sonify_spectrum_via_columnwise_fft(stft, hop_size, n_fft):
    
    sonified_spectrum = np.zeros(int((stft.shape[1]-1) * hop_size + n_fft))
    
    
    # Iterate through each frame in the STFT
    for t in range(stft.shape[1]):
        
        # Inverse STFT: Reconstruct the time-domain signal for each frame
        frame = np.fft.irfft(stft[:, t], n=n_fft)
        
        # Add to the reconstructed audio
        start = t * hop_size
        end = start + n_fft
        sonified_spectrum[start:end] += frame
    
    # Normalize the reconstructed audio to prevent clipping
    sonified_spectrum /= np.max(np.abs(sonified_spectrum))
        
    return sonified_spectrum

In [ ]:
sonified_spectrum = sonify_spectrum_via_columnwise_fft(X, H, N)

print('Sonified Spectrum:')
ipd.display(ipd.Audio(sonified_spectrum, rate=Fs))

## Sonification via bin-wise sinusoidals

In [ ]:
def sonify_spectrum_via_binwise_sinusoidals(stft, hop_size, n_fft, sample_rate):
    
    # Initialize an array to store the reconstructed audio
    sonified_spectrum = np.zeros(int((stft.shape[1]-1) * hop_size + n_fft))

    
    # Iterate through each column in the STFT
    for t in range(stft.shape[1]):
        
        # Initialize the frame's audio as all zeros
        frame_audio = np.zeros(n_fft)
        
        # Iterate through each frequency bin in the STFT
        for f in range(stft.shape[0]):
            
            # Get magnitude and phase for the current bin
            magnitude = np.abs(stft[f, t])
            phase = np.angle(stft[f, t])
            
            # Calculate the frequency in Hz for the current bin
            frequency = f * sample_rate / n_fft
    
            # Generate a sinusoid for the current bin
            sinusoid = magnitude * np.sin(2 * np.pi * frequency * (np.arange(n_fft) / sample_rate) + phase)
            
            # Add the sinusoid to the frame's audio
            frame_audio += sinusoid
        
        # Overlap and add the frame's audio to the reconstructed audio
        start = t * hop_size
        end = start + n_fft
        if end <= len(sonified_spectrum):
            sonified_spectrum[start:end] += np.real(frame_audio)  # Use real part for audio synthesis
    
    # Normalize the reconstructed audio to prevent clipping
    sonified_spectrum /= np.max(np.abs(sonified_spectrum))

    return sonified_spectrum

In [ ]:
sonified_spectrum = sonify_spectrum_via_binwise_sinusoidals(X, H, N, Fs)

print('Sonified Spectrum:')
ipd.display(ipd.Audio(sonified_spectrum, rate=Fs))

In [ ]:
S = librosa.feature.melspectrogram(y=x, sr=Fs, n_mels=2048,
                                    fmax=int(Fs/2))
fig, ax = plt.subplots()
S_dB = librosa.power_to_db(S, ref=np.max)
img = librosa.display.specshow(S_dB, x_axis='time',
                         y_axis='mel', sr=Fs,
                        fmax=8000, ax=ax)
fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set(title='Mel-frequency spectrogram')


n_fft = 2048  # FFT window size (adjust as needed)
hop_length = 512  # Hop size (adjust as needed)

# Inverse Mel Filterbank
power_spectrogram = librosa.feature.inverse.mel_to_stft(S)

# Inverse STFT to obtain the time-domain audio signal
audio_signal = librosa.istft(power_spectrogram, hop_length=hop_length, win_length=n_fft)

# Normalize the reconstructed audio to prevent clipping
audio_signal /= np.max(np.abs(audio_signal))
ipd.display(ipd.Audio(audio_signal, rate=Fs))
